# JS 如何通过事件循环机制（EventLoop）实现异步执行

JS 是单线程同步语言，却能实现异步执行，靠的就是事件循环机制。

在讲事件循环机制之前先插入几个概念。


### 运行时（runtime）

JS 运行时全称 JS 运行时环境，是 JS 代码的执行环境，由 JS 引擎、事件循环机制、回调队列和宿主 APIs 组成。

JS 引擎通过调用栈和堆执行代码和管理内存。

- 调用栈：
- 内存堆：

宿主是浏览器、Node


执行上下文被创建之后会被压入调用栈，调用栈是跟踪代码执行流程的数据结构，当代码开始执行的时候会首先创建全局执行上下文并入栈，
执行到函数的时候会创建函数执行上下文并入栈，调用栈只执行栈顶的执行上下文代码，栈顶函数执行完就出栈，继续执行调用栈的栈顶函数。

在执行上下文里，基本类型的变量值存在环境记录里，引用类型和函数的值存在堆内存里，环境记录里存的是堆的内存地址。
